In [32]:
import requests
from bs4 import BeautifulSoup
import sqlite3 as lite
import pandas as pd
import re
import time

In [8]:
Addr = "https://www.larvalabs.com/cryptopunks/details" # Punk number
PunkNo = '10'
Punk = 10
PunkRange = (Punk*(10))

PunkList = list(range(0,PunkRange+1,1))

In [9]:
BaseStr = "https://www.larvalabs.com/cryptopunks/details/"
con = lite.connect('CryptoPunk.db')

In [15]:
with con:
        cur=con.cursor()
        cur.execute("DROP TABLE IF EXISTS PunkTrades") 
       
        cur.execute("""CREATE TABLE PunkTrades(PunkID text, TType text, TFrom text, TTo text, TAmt int, TDate text)""");
        listing = []
        for i in PunkList:
            PunkNo = str(i)
            print('Processing Punk No.:' + PunkNo)
            page = requests.get(BaseStr + PunkNo)             
            # Getting page HTML through request
            soup = BeautifulSoup(page.content, 'html.parser') 
            # Parsing content using beautifulsoup
            #< Complete reading of Crypto trades for this Punk looping through all the trades>#
            
            table = soup.find('table', attrs={'class':'table'})
            rows = table.find_all('tr')
            time.sleep(1)
            for row in rows:
                cols = row.find_all('td')
                if cols:
                    cols = [ele.text.strip() for ele in cols]
                    # print(cols)
                    PunkID = PunkNo
                    TType = cols[0]
                    TFrom = cols[1]
                    TTo = cols[2]
                    TAmtList = [x[0] for x in re.findall('\$((\d+)(?:[\.,]\d+))', cols[3])]
                    TAmtS = ' '.join(TAmtList).replace(',','')
                    if TAmtS == '':
                        TAmtS = 0
                    TAmt = int(float(TAmtS))
                    type(TAmt)
                    # TAmt = cols[3]
                    # print(TAmt)
                    TDate = cols[4]


                    row1 = (PunkID, TType, TFrom, TTo, TAmt, TDate)
                    cur.execute('INSERT OR IGNORE INTO PunkTrades VALUES (?,?,?,?,?,?)', row1)

Processing Punk No.:0
Processing Punk No.:1
Processing Punk No.:2
Processing Punk No.:3
Processing Punk No.:4
Processing Punk No.:5
Processing Punk No.:6
Processing Punk No.:7
Processing Punk No.:8
Processing Punk No.:9
Processing Punk No.:10
Processing Punk No.:11
Processing Punk No.:12
Processing Punk No.:13
Processing Punk No.:14
Processing Punk No.:15
Processing Punk No.:16
Processing Punk No.:17
Processing Punk No.:18
Processing Punk No.:19
Processing Punk No.:20
Processing Punk No.:21
Processing Punk No.:22
Processing Punk No.:23
Processing Punk No.:24
Processing Punk No.:25
Processing Punk No.:26
Processing Punk No.:27
Processing Punk No.:28
Processing Punk No.:29
Processing Punk No.:30
Processing Punk No.:31
Processing Punk No.:32
Processing Punk No.:33
Processing Punk No.:34
Processing Punk No.:35
Processing Punk No.:36
Processing Punk No.:37
Processing Punk No.:38
Processing Punk No.:39
Processing Punk No.:40
Processing Punk No.:41
Processing Punk No.:42
Processing Punk No.:4

#### Report the punk with the highest price

In [17]:
Qry = ('''SELECT PunkID, MAX(TAmt) AS Price
        FROM PunkTrades WHERE TType = 'Sold' ''')

Q1 = pd.read_sql_query(Qry, con)
Q1

,PunkID,Price
0,53,432676


#### Report which punk was traded the most.

In [18]:
Qry = (''' SELECT PunkID, COUNT(TType) AS Trade_count
            FROM PunkTrades
            WHERE TType == 'Sold' AND TAmt>0
            GROUP BY PunkID ORDER BY Trade_count DESC LIMIT 1 ''')

Q2 = Q1 = pd.read_sql_query(Qry, con)
Q2

,PunkID,Trade_count
0,53,3


#### Average price by day in the DB

In [31]:
Qry = (''' SELECT AVG(TAmt) AS AverageP, TDate
            FROM PunkTrades WHERE TAmt>0 AND TType == 'Sold' GROUP BY TDate ''')

Q3 = pd.read_sql_query(Qry, con)
Q3

,AverageP,TDate
0,5155.0,"Apr 06, 2019"
1,432676.0,"Aug 23, 2021"
2,26612.0,"Feb 01, 2021"
3,42803.0,"Feb 19, 2021"
4,4138.0,"May 28, 2020"
5,2822.0,"Nov 30, 2018"
6,36305.0,"Nov 30, 2020"
7,6898.0,"Sep 06, 2020"


#### Which owner has the most valuable portfolio of Crypto Punks?

In [37]:
Qry = (''' SELECT TTo as PunkOwner, SUM(TAmt) AS portfolio
            FROM(SELECT PunkID, TTo, TAmt, MAX(TDate) AS TDate
            FROM PunkTrades WHERE TType = 'Sold' GROUP BY PunkID)
            GROUP by TTo ORDER BY portfolio DESC LIMIT 1 ''')

Q4 = pd.read_sql_query(Qry, con)
Q4

,PunkOwner,portfolio
0,jmg,42803
